In [15]:
# Import libraries
from pymongo import MongoClient
from pprint import pprint

# Create MongoDB client
client = MongoClient('mongodb://localhost:27017/')

# List all databases
print(client.list_database_names())

# Set the database and collection
db = client.uk_food
establishments = db.establishments

# List collections in the database
print(db.list_collection_names())

# Print a single document from the establishments collection
pprint(establishments.find_one())


ModuleNotFoundError: No module named 'pymongo'

In [ ]:
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

establishments.insert_one(new_restaurant)


In [ ]:
result = establishments.find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, {"BusinessTypeID": 1, "BusinessType": 1})
pprint(result)


In [ ]:
establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": result["BusinessTypeID"]}}
)


In [ ]:
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of establishments in Dover: {dover_count}")


In [ ]:
establishments.delete_many({"LocalAuthorityName": "Dover"})


In [ ]:
dover_count_after = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of establishments in Dover after deletion: {dover_count_after}")


In [ ]:
establishments.update_many(
    {},
    [{"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}, "geocode.longitude": {"$toDouble": "$geocode.longitude"}}}]
)


In [ ]:
establishments.update_many(
    {"RatingValue": {"$type": "string"}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)


In [ ]:
hygiene_20 = establishments.find({"scores.Hygiene": 20})
for establishment in hygiene_20:
    pprint(establishment)


In [ ]:
london_establishments = establishments.find({"LocalAuthorityName": {"$regex": "London"}, "RatingValue": {"$gte": 4}})
for establishment in london_establishments:
    pprint(establishment)


In [ ]:
penang_flavours = establishments.find_one({"BusinessName": "Penang Flavours"})
lat, lon = penang_flavours["geocode"]["latitude"], penang_flavours["geocode"]["longitude"]

top_5_establishments = establishments.find({
    "RatingValue": 5,
    "geocode.latitude": {"$gte": lat - 0.01, "$lte": lat + 0.01},
    "geocode.longitude": {"$gte": lon - 0.01, "$lte": lon + 0.01}
}).sort("scores.Hygiene", 1).limit(5)

for establishment in top_5_establishments:
    pprint(establishment)


In [ ]:
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]
top_local_authorities = list(establishments.aggregate(pipeline))

pprint(top_local_authorities)
